In [1]:
%matplotlib inline

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# What is a neural network

Neural network is a collection of neurons that are connected by layers. Each neuron is a small
computing unit that performs simple calculations to collectively solve a problem. They are
organized in layers. There are 3 types of layers: input layer, hidden layer and
outter layer. Each layer contains a number of neurons, except for the input layer. Neural networks mimic the way a human brain processes information.

<img alt="Diagram showing different types of layers in a neural network" src="figs/4-model-1.png" />


## Components of a neural network

- **Activation function** determines whether a neuron should be activated or not. The computations that happen in a neural network include applying an activation function. If a neuron activates, then it means the input is important. The are different kinds of activation functions. The choice of which activation function to use depends on what you want the output to be. Another important role of an activation function is to add non-linearity to the model.
  - _Binary_ used to set an output node to 1 if function result is positive and 0 if the function result is negative. $f(x)= {\small \begin{cases} 0, & \text{if } x < 0\\ 1, & \text{if } x\geq 0\\ \end{cases}}$
  - _Sigmod_ is used to predict the probability of an output node being between 0 and 1. $f(x) = {\large \frac{1}{1+e^{-x}}} $
  - _Tanh_ is used to predict if an output node is between 1 and -1. Used in classification use cases. $f(x) = {\large \frac{e^{x} - e^{-x}}{e^{x} + e^{-x}}} $
  - _ReLU_ used to set the output node to 0 if fuction result is negative and keeps the result value if the result is a positive value. $f(x)= {\small \begin{cases} 0, & \text{if } x < 0\\ x, & \text{if } x\geq 0\\ \end{cases}}$


- **Weights** influence how well the output of our network will come close to the expected output value. As an input enters the neuron, it gets multiplied by a weight value and the resulting output is either observed or passed to the next layer in the neural network. Weights for all neurons in a layer are organized into one tensor


- **Bias** makes up the difference between the activation function's output and its intended output. A low bias suggests that the network is making more assumptions about the form of the output, whereas a high bias value makes less assumptions about the form of the output.


<img alt="Diagram showing neural computation" src="figs/4-model-2.png" />

We can say that an output $y$ of a neural network layer with weights $W$ and bias $b$ is computed as a summation of the inputs multiplied by the weights plus the bias $x = \sum{(weights * inputs) + bias} $, where $f(x)$ is the activation function.


## Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data.
The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to
build your own neural network. Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.


## Get Device for Training

We want to be able to train our model on a hardware accelerator like the GPU,
if it is available. Let's check to see if
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) is available, else we
continue to use the CPU.


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Define the Class

1. We define our neural network by subclassing `nn.Module`
2. initialize the neural network layers in `__init__`.
3. Every `nn.Module` subclass implements the operations on input data in the `forward` method.


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`, and print
its structure.


In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model's `forward`,
along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!


Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output. .
We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


## Model Layers

Let's break down the layers in the FashionMNIST model. To illustrate it, we
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as
we pass it through the network.


In [6]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its stored weights and biases.


In [9]:
layer1 = nn.Linear(in_features=28 * 28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce _nonlinearity_, helping neural networks
learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our
linear layers, but there's other activations to introduce non-linearity in your model.


![](figs/relu.webp)


In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2984,  0.5974,  0.1858,  0.0534, -0.0356, -0.1865,  0.0461,  0.0730,
         -0.2088, -0.9550,  0.0160, -0.2157, -0.0354, -0.3729,  0.2369,  0.1836,
         -0.1148,  0.2003,  0.0109, -0.2129],
        [ 0.2080,  0.4216,  0.2407,  0.2661, -0.3295,  0.0119, -0.0370,  0.6470,
         -0.5641, -1.0290,  0.3032, -0.3217,  0.2718, -0.4592,  0.3068, -0.2711,
         -0.1246,  0.7126,  0.2643, -0.2022],
        [-0.0026,  0.4888,  0.3803,  0.3746, -0.4222, -0.3190, -0.1408,  0.0427,
         -0.1878, -0.9172,  0.2667, -0.2904, -0.2068, -0.2338,  0.1762, -0.1347,
          0.2351,  0.4710,  0.2873, -0.2718]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.5974, 0.1858, 0.0534, 0.0000, 0.0000, 0.0461, 0.0730, 0.0000,
         0.0000, 0.0160, 0.0000, 0.0000, 0.0000, 0.2369, 0.1836, 0.0000, 0.2003,
         0.0109, 0.0000],
        [0.2080, 0.4216, 0.2407, 0.2661, 0.0000, 0.0119, 0.0000, 0.6470, 0.0000,
         0.0000, 0.3032, 0.0000, 0.2718, 0.0000, 0.30

### nn.Sequential

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like `seq_modules`.


In [11]:
seq_modules = nn.Sequential(flatten, layer1, nn.ReLU(), nn.Linear(20, 10))
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

### nn.Softmax

The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values
[0, 1] representing the model's predicted probabilities for each class. `dim` parameter indicates the dimension along
which the values must sum to 1.


![](figs/softmax.jpeg)


In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters

Many layers inside a neural network are _parameterized_, i.e. have associated weights
and biases that are optimized during training. Subclassing `nn.Module` automatically
tracks all fields defined inside your model object, and makes all parameters
accessible using your model's `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.


In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0302,  0.0175,  0.0178,  ..., -0.0337, -0.0046, -0.0133],
        [ 0.0045, -0.0192, -0.0137,  ...,  0.0151,  0.0074,  0.0232]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0355, 0.0271], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0394, -0.0410,  0.0408,  ..., -0.0025,  0.0021,  0.0255],
        [ 0.0259, -0.0254, -0.0322,  ...,  0.0289,  0.0116,  0.0127]],
       device='cuda:0', grad_fn=<Slic

## Further Reading

- [torch.nn API](https://pytorch.org/docs/stable/nn.html)
